In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [1]:
# load packages
import json
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
print('packages loaded')

In [3]:
%%time
# review data
revfile = open('/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json')
data=[]
for count, line in enumerate(revfile):
    data.append(json.loads(line))
    if count == 4500000:
        revdata = pd.DataFrame(data)
        revdata.to_csv('reviewdf1.csv', index=False)
        del revdata
        gc.collect
        data = []
    else:
        continue
revdata = pd.DataFrame(data)
revdata.to_csv('reviewdf2.csv', index=False)
revfile.close()


In [4]:
del revdata, data
gc.collect()


In [5]:
# load reviews, concatenate to single df
revdf1 = pd.read_csv('reviewdf1.csv')
revdf2 = pd.read_csv('reviewdf2.csv')
revdf = pd.concat([revdf1,revdf2], ignore_index=True)
revdf.info()

In [6]:
# drop selected cols
revdf = revdf.drop(['useful','funny','cool'], axis=1)
revdf.info()

In [7]:
revdf.head()

In [8]:
# business data
data_file = open('/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json')
data = []
for line in data_file:
    data.append(json.loads(line))
df = pd.DataFrame(data)
data_file.close()
del data
gc.collect()

df.info()

In [9]:
df.head()

In [10]:
# drop selected columns
df = df.drop(df.columns[[6,7,10,11,13]], axis=1)
df.info()

In [11]:
# check what kinds of businesses have na
print(df.isna().sum())
df[df.categories.isna()]

In [12]:
# drop rows with no categories listed
df = df.dropna(axis=0)
# select restaurants 
restaurants = df.loc[:, 'categories'].str.contains('Restaurants') # bool, restaurant rows
df = df.loc[restaurants,:]
df.info()

In [13]:
# select restaurant reviews only
revdf = revdf[revdf.business_id.isin(df.business_id.values)]
print(revdf.info())
revdf.head()

In [14]:
revdf.business_id.unique().shape
revdf.to_csv('restaurantrev.csv',index=False)
df.to_csv('restaurants.csv', index=False)


<a href="./restaurantrev.csv"> Download restaurantreview File </a>


In [15]:
os.remove('./reviewdf1.csv')
os.remove('./reviewdf2.csv')

In [16]:
plt.hist(revdf.stars)
#plt.hist(revdf.stars, histtype='step',density=True, cumulative=True)
revdf.value_counts('stars')

In [2]:
# user data
data_file = open('/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json')
data = []
for line in data_file:
    data.append(json.loads(line))
df = pd.DataFrame(data)
#df.to_csv('userdf.csv', index=False)
data_file.close()
del data
gc.collect()

In [4]:
df.head()

In [5]:
# remove unneeded columns 
cols = df.columns[4:7].append(df.columns[11::])
cols = cols.append(df.columns[8:10])
print(cols)
# save to csv file
df.drop(cols, axis=1).to_csv('userdf.csv', index=False)

In [6]:
del df
gc.collect()

In [7]:
# load user csv 
userdf = pd.read_csv('./userdf.csv', parse_dates=['yelping_since'])
userdf.head()

In [8]:
userdf.info()

In [9]:
userdf.describe()